# Successful calls start and end times

Find all the start and end times for each of the successful calls in the scenarios 1, 8, 9, 10, 11.

In [1]:
import sys
from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
import os
import json
import logging
import helperFunctions as hf
import numpy as np

hf.setup()

outputFolder = "output_folder/"
outputFile = outputFolder + "SuccessfulCallsStartAndEnd.csv"

client = MongoClient(hf.getConnectionString())
database=client["observertc-reports"]
calls=database["calls"]

if not os.path.exists(outputFolder):

   # Create a new directory because it does not exist
   os.makedirs(outputFolder)
   logging.info(f"The directory \"{outputFolder}\" is created!")

In [2]:
df = pd.DataFrame(columns=["scenario", "client", "start", "end"])

uniqueCallsDf = pd.read_csv(outputFolder + "UniqueCallsAndOutcomes.csv")

logging.info(f"Shape of uniqueCallsDf: {uniqueCallsDf.shape}")

# filter for calls with state success
successCallsDf = uniqueCallsDf.loc[uniqueCallsDf["logging_type"] == "COMMAND_SESSION_SUCCESS"]

logging.info(f"Shape of successful calls: {successCallsDf.shape}")

for i in [1, 8, 9, 10, 11]:

  s = hf.scenarios[i-1]["scenario"]
  a = hf.scenarios[i-1]["alice"]
  b = hf.scenarios[i-1]["bob"]
  logging.info(f"Starting on scenario {s}, alice: {a}, bob: {b}")

  # filter for calls with scenario_type s
  scenarioCallsDf = successCallsDf.loc[successCallsDf["scenario_type"] == s]

  # get list of room_ids in scenarioCallsDf
  roomIds = scenarioCallsDf["room_id"].unique()

  for roomId in roomIds:

    aStart = calls.find_one({"room_id": str(roomId), "client_username": str(a), "state": "call_in_progress"})["timestamp"]
    aEnd = calls.find_one({"room_id": str(roomId), "client_username": str(a), "state": "call_ended"})["timestamp"]
    dataA = pd.DataFrame(columns=["scenario", "client", "start", "end"], data=[[s, a, aStart, aEnd]])

    bStart = calls.find_one({"room_id": str(roomId), "client_username": str(b), "state": "call_in_progress"})["timestamp"]
    bEnd = calls.find_one({"room_id": str(roomId), "client_username": str(b), "state": "call_ended"})["timestamp"]
    dataB = pd.DataFrame(columns=["scenario", "client", "start", "end"], data=[[s, b, bStart, bEnd]])

    df = pd.concat([df, dataA, dataB], ignore_index=True)

df.to_csv(outputFile, index=False, header=True, encoding='utf-8', mode='w')


2023-02-05 12:15:04 INFO     Shape of uniqueCallsDf: (19222, 7) 
2023-02-05 12:15:04 INFO     Shape of successful calls: (19064, 7) 
2023-02-05 12:15:04 INFO     Starting on scenario 1, alice: c1-Normal, bob: d1-Normal 
2023-02-05 12:19:02 INFO     Starting on scenario 8, alice: c1-Normal, bob: d3-TorEurope 
2023-02-05 12:22:36 INFO     Starting on scenario 9, alice: c3-TorEurope, bob: d1-Normal 
2023-02-05 12:26:03 INFO     Starting on scenario 10, alice: c1-Normal, bob: d4-TorScandinavia 
2023-02-05 12:28:38 INFO     Starting on scenario 11, alice: c4-TorScandinavia, bob: d1-Normal 
